In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
from datetime import date

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

import numpy as np
from epiweeks import Week, Year
from datetime import date, timedelta

In [ ]:
from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.systems import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_text

In [ ]:
nt = Notifica()
nt.load(f"notifica", compress=False)

In [ ]:
for col in nt.tables['municipios'].columns:
    if col in nt.df.columns:
        del nt.df[col]

In [ ]:
nt.df['ibge_unidade_notifica'].unique()

In [ ]:
nt.df.loc[nt.df['ibge_unidade_notifica'].apply(str).str[:2]!='41']

In [ ]:
# nt.df.loc[nt.df['ibge_residencia'].apply(str).str[:2]!='41', 'ibge_residencia'] = 999999
# nt.df = nt.df.drop(index=nt.df.loc[nt.df['ibge_residencia'].apply(str).str[:2]!='41'].index)
nt.df = nt.df.drop(index=nt.df.loc[nt.df['ibge_unidade_notifica'].apply(str).str[:2]!='41'].index)
# nt.df.loc[nt.df['ibge_unidade_notifica'].apply(str).str[:2]!='41', 'ibge_unidade_notifica'] = 999999

In [ ]:
nt.df.drop(index=nt.df.loc[nt.df['data_notificacao'].isna()].index, inplace=True)

In [ ]:
# nt.bk = nt.df.copy()

In [ ]:
# nt.tables['exame'].set_index('id')['exame'].loc[[167, 184, 187, 188, 189, 190, 191]]

In [ ]:
# nt.tables['metodo'].set_index('id')

In [ ]:
# del nt.df
# nt.df = nt.bk

In [ ]:
# nt.df = nt.df.loc[nt.df['exame'].isin([167])]#, 187, 188, 189, 190, 191])]
# nt.df.groupby('metodo')[['id']].count().rename(index=nt.tables['metodo'].set_index('id')['metodo'])
# nt.df = nt.df.loc[nt.df['metodo']==2]

In [ ]:
# nt.df.groupby('')[['id']].count().rename(index=nt.tables['metodo'].set_index('id')['metodo'])

In [ ]:
# nt.df = nt.df.loc[(nt.df["data_notificacao"] >= pd.to_datetime(date(2022,1,5))) & (nt.df["data_notificacao"] <= pd.to_datetime(date(2022,1,11)))]
# nt.df

In [ ]:
# nt.df.groupby('exame')[['id']].count().rename(index=nt.tables['exame'].set_index('id')['exame']).sort_values('id', ascending=False)

In [ ]:
nt.df['ano'] = nt.df['data_notificacao'].apply(lambda x: Week.fromdate(x).year)
nt.df['se'] = nt.df['data_notificacao'].apply(lambda x: Week.fromdate(x).week)

In [ ]:
nt.df.loc[nt.df['classificacao_final']==4,'classificacao_final'] = 1
nt.df.loc[nt.df['classificacao_final']==5,'classificacao_final'] = 3

In [ ]:
nt.df.groupby(['classificacao_final'])[['id']].count()#.rename(index=nt.tables['classificacao_final'].set_index('id')['classificacao_final'])

In [ ]:
ses = []
for year in [2020,2021,2022]:
    for se in Year(year).iterweeks():
        if (se.startdate() <= hoje.date()) & (se.startdate() >= date(2020,3,1)):
#         if (se.startdate() <= hoje.date()) & (se.startdate() >= date(2021,12,1)):
            ses.append((se.year,se.week))

In [ ]:
mun = nt.tables['municipios'].copy()
mun = mun.loc[mun['uf'].isin(['PR','IG'])]
mun = mun[['uf','macro','rs','regional','ibge','municipio']]
nt.df = nt.df.join(mun.set_index('ibge'),on='ibge_unidade_notifica').rename(columns={'ibge_unidade_notifica':'ibge'})

In [ ]:
total_mun = nt.df.groupby(['uf','macro','rs','regional','ibge','municipio','ano','se','data_notificacao'])[['id']].count().unstack(['ano','se','data_notificacao']).fillna(0).astype(int).sort_index(axis=1,level=1).droplevel(axis=1,level=0)
# ses = [ se for se in ses if se in total_mun.columns ]
# total_mun = total_mun.reindex(ses,axis=1).fillna(0).astype(int)

confirmados_mun = nt.df.loc[nt.df['classificacao_final']==2].groupby(['uf','macro','rs','regional','ibge','municipio','ano','se','data_notificacao'])[['id']].count().unstack(['ano','se','data_notificacao']).fillna(0).astype(int).sort_index(axis=1,level=1).droplevel(axis=1,level=0)
# print(confirmados_mun.columns)
# print(ses)
# confirmados_mun = confirmados_mun.reindex(ses,axis=1).fillna(0).astype(int)

positividade_mun = confirmados_mun / total_mun

In [ ]:
# raise
# pd.concat({'1.Total':total_mun, '2.Confirmados':confirmados_mun, '3.Positividade':positividade_mun },names=['ind'],axis=0).reorder_levels(['uf','macro','rs','regional','ibge','municipio','ind'],axis=0).sort_index(axis=0,level=['rs','ibge'])

In [ ]:
total_rs = total_mun.groupby(['rs','regional']).sum().fillna(0).astype(int)
confirmados_rs = confirmados_mun.groupby(['rs','regional']).sum().fillna(0).astype(int)
positividade_rs = confirmados_rs / total_rs

In [ ]:
# pd.concat({'1.Total':total_rs, '2.Confirmados':confirmados_rs, '3.Positividade':positividade_rs },names=['ind'],axis=0).reorder_levels(['rs','regional','ind'],axis=0).sort_index(axis=0,level=['rs'])

In [ ]:
total_macro = total_mun.groupby(['macro']).sum().fillna(0).astype(int)
confirmados_macro = confirmados_mun.groupby(['macro']).sum().fillna(0).astype(int)
positividade_macro = confirmados_macro / total_macro

In [ ]:
# pd.concat({'1.Total':total_macro, '2.Confirmados':confirmados_macro, '3.Positividade':positividade_macro },names=['ind'],axis=0).reorder_levels(['macro','ind'],axis=0).sort_index(axis=0,level=['macro'])

In [ ]:
total = total_mun.groupby(['uf']).sum().fillna(0).astype(int)
confirmados = confirmados_mun.groupby(['uf']).sum().fillna(0).astype(int)
positividade = confirmados / total

In [ ]:
a = pd.concat({'1.Total':total, '2.Confirmados':confirmados, '3.Positividade':positividade },names=['ind'],axis=0).reorder_levels(['uf','ind'],axis=0).sort_index(axis=0,level=['uf']).droplevel(axis=0,level=0).T
ax = a[['1.Total','2.Confirmados']].rolling(7).mean().plot()
a[['3.Positividade']].rolling(7).mean().plot(secondary_y=True, ax=ax, ls=':', figsize=(20,12))

In [ ]:
total.loc[:,(2021,40):].sum(1)

In [ ]:
writer = pd.ExcelWriter(join(default_output,"positividade.xlsx"))

total.to_excel(writer,'total')
confirmados.to_excel(writer,'confirmados')
positividade.to_excel(writer,'positividade')

total_macro.to_excel(writer,'total_macro')
confirmados_macro.to_excel(writer,'confirmados_macro')
positividade_macro.to_excel(writer,'positividade_macro')

total_rs.to_excel(writer,'total_rs')
confirmados_rs.to_excel(writer,'confirmados_rs')
positividade_rs.to_excel(writer,'positividade_rs')

total_mun.to_excel(writer,'total_mun')
confirmados_mun.to_excel(writer,'confirmados_mun')
positividade_mun.to_excel(writer,'positividade_mun')

writer.save()